# 数据集构建

In [1]:
# 导入相关模块
import warnings
import numpy as np
import pandas as pd
import random
import pickle
import re
warnings.filterwarnings('ignore')

In [2]:
# 读取 Excel 文件
df = pd.read_csv('../data/raw/raw_data.csv')
df.replace(np.nan, 'None', inplace=True)  # 将所有 '' 替换为nan
print(df.shape)
# 删除重复记录
df = df.drop_duplicates(subset=['person_id', 'pri_code', 'disd_c1', 'disd_c2', 'disd_c3', 'disd_c4', 'disd_c5', 'disd_c6',
 'disd_c7', 'disd_c8', 'disd_c9', 'disd_c10', 'disd_c11', 'disd_c12', 'disd_c13',
 'disd_c14', 'disd_c15'])
print(df.shape)

(348539, 25)
(278629, 25)


In [3]:


# 使用 value_counts() 计算每个值的出现次数
value_counts = df['person_id'].value_counts()

# 通过布尔索引只保留出现次数在50次以下的数据
df = df[df['person_id'].isin(value_counts[value_counts <= 50].index)]
df.shape

(271980, 25)

In [4]:
# 假设你要将 'old_column_name' 更名为 'new_column_name'
df.rename(columns={'person_id': 'SUBJECT_ID', 'user': 'HADM_ID', 'date': 'ADMITTIME'}, inplace=True)
df['HADM_ID'] = df.index

In [5]:
col_index = df.columns.values  # 列名： 数组形式<class 'numpy.ndarray'>['姓名' '性别' '年龄' '住址' '爱好']
print('列名是：%s' % col_index)

列名是：['SUBJECT_ID' 'HADM_ID' 'record_num' 'ADMITTIME' 'bir' 'age' 'gender'
 'ocp' 'mary' 'pri_code' 'disd_c1' 'disd_c2' 'disd_c3' 'disd_c4' 'disd_c5'
 'disd_c6' 'disd_c7' 'disd_c8' 'disd_c9' 'disd_c10' 'disd_c11' 'disd_c12'
 'disd_c13' 'disd_c14' 'disd_c15']


In [6]:
Adm_ = df.copy()
Adm_.drop(columns=['disd_c1', 'disd_c2', 'disd_c3', 'disd_c4', 'disd_c5', 'disd_c6',
 'disd_c7', 'disd_c8', 'disd_c9', 'disd_c10', 'disd_c11', 'disd_c12', 'disd_c13',
 'disd_c14', 'disd_c15'], inplace=True)

In [7]:
Adm_

,SUBJECT_ID,HADM_ID,record_num,ADMITTIME,bir,age,gender,ocp,mary,pri_code
0,1,0,5046540.0,2022-03-05,1974-04-16,47,2,27.0,20.0,G47.9
1,2,1,1678982.0,2018-10-29,1968-06-06,50,2,17.0,20.0,I67.8
2,3,2,None,2020-04-09,1938-09-03,81,1,80.0,20.0,J18.9
3,3,3,46188.0,2020-05-13,1938-09-03,81,1,80.0,20.0,F01.1
4,3,4,57492.0,2020-06-12,1938-09-03,81,1,80.0,20.0,J69.0
...,...,...,...,...,...,...,...,...,...,...
348534,102408,348534,5712574.0,2022-12-07,1931-01-01,91,2,80.0,20.0,I63.3
348535,102408,348535,5735758.0,2022-12-19,1931-01-01,91,2,80.0,20.0,F41.1
348536,102408,348536,5804901.0,2023-01-13,1931-01-01,92,2,80.0,20.0,J15.8
348537,102408,348537,5932560.0,2023-01-31,1931-01-01,92,2,80.0,20.0,F01.2


In [8]:
# 列出你感兴趣的列名
columns_of_interest = ['pri_code', 'disd_c1', 'disd_c2', 'disd_c3', 'disd_c4', 'disd_c5', 'disd_c6', 'disd_c7', 'disd_c8', 'disd_c9', 'disd_c10', 'disd_c11', 'disd_c12', 'disd_c13', 'disd_c14', 'disd_c15']

# 创建一个空集合用于存储唯一值
unique_values = set()

# 遍历列名，并将每列的唯一值添加到集合中
for column in columns_of_interest:
    unique_values.update(pd.unique(df[column]))

print(len(unique_values))


5129


In [9]:
del_result = [value for value in unique_values if value.endswith('.')]
print(del_result)

[]


In [10]:
id_counts = Adm_['SUBJECT_ID'].value_counts()
# 输出counts大于1的人数
result = id_counts[id_counts > 1].index
print(len(id_counts), len(result))

102309 43401


In [11]:
df

,SUBJECT_ID,HADM_ID,record_num,ADMITTIME,bir,age,gender,ocp,mary,pri_code,...,disd_c6,disd_c7,disd_c8,disd_c9,disd_c10,disd_c11,disd_c12,disd_c13,disd_c14,disd_c15
0,1,0,5046540.0,2022-03-05,1974-04-16,47,2,27.0,20.0,G47.9,...,None,None,None,None,None,None,None,None,None,None
1,2,1,1678982.0,2018-10-29,1968-06-06,50,2,17.0,20.0,I67.8,...,None,None,None,None,None,None,None,None,None,None
2,3,2,None,2020-04-09,1938-09-03,81,1,80.0,20.0,J18.9,...,E87.8,D64.9,None,None,None,None,None,None,None,None
3,3,3,46188.0,2020-05-13,1938-09-03,81,1,80.0,20.0,F01.1,...,None,None,None,None,None,None,None,None,None,None
4,3,4,57492.0,2020-06-12,1938-09-03,81,1,80.0,20.0,J69.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348534,102408,348534,5712574.0,2022-12-07,1931-01-01,91,2,80.0,20.0,I63.3,...,M80.8,N19.x,D38.1,E78.2,D64.9,R80.x,None,None,None,None
348535,102408,348535,5735758.0,2022-12-19,1931-01-01,91,2,80.0,20.0,F41.1,...,N19.x,None,None,None,None,None,None,None,None,None
348536,102408,348536,5804901.0,2023-01-13,1931-01-01,92,2,80.0,20.0,J15.8,...,I38.x,E87.8,D64.9,I69.3,F01.9,I10.x,None,None,None,None
348537,102408,348537,5932560.0,2023-01-31,1931-01-01,92,2,80.0,20.0,F01.2,...,I65.0,I10.x,G45.8,D64.9,E77.8,R09.0,E87.6,N39.0,None,None


In [12]:
import pandas as pd
import numpy as np

# 创建子数据框
subset_columns = ['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ICD10_CODE', 'Sort']
subset_dfs = [pd.DataFrame(columns=subset_columns) for _ in range(16)]

# 将特定值赋为空值
df.replace('None', np.nan, inplace=True)

# 填充子数据框
for i, subset_df in enumerate(subset_dfs):
    column_name = 'pri_code' if i == 0 else f'disd_c{i}'
    subset_df[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'ICD10_CODE']] = df[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', column_name]]
    subset_df['Sort'] = i

# 合并子数据框
Full_data = pd.concat(subset_dfs, axis=0, ignore_index=True)
# 将NaN替换为一个标记值（例如空字符串），以便使用startswith()方法
Full_data['ICD10_CODE'] = Full_data['ICD10_CODE'].fillna('None')
# 删除非诊断节点
del_icd = del_result + ['Z54', 'T91', 'T00', 'E14', 'T90', 'T92', 'T01', 'T06', 'T08', 'T02', 'T93', 'T29', 'B24', 'B23', 'R96', 'T09', 'M8000', 'M9530', 'T12', 'T94', 'T97', 'T13', 'Y49', 'T11', 'M8940', 'D34 x', 'B22', 'T98', 'M8010', 'M8070', 'M8140', 'O83', 'T10', 'M9950', 'M8950', 'M8854', 'M9540', 'U88', 'F00', 'B59+J', 'C97', 'M8810', 'Y41', 'k60', 'Y57', 'Y16', 'M9100', 'W79', 'M8170', 'M8890', 'M9121', 'M8851', 'O84', 'M9210', 'T95', 'T96', 'M8850', 'BWP10', 'M9080', 'M8720', 'BFY20', 'T04', 'M8211', 'I10xx', 'N40xx', 'M9590', 'M8311', 'M8750', 'M8895', 'M9120', 'M8210', 'M8120', 'M8050', 'M9960', 'M8561', 'T03', 'X70', 'M8272', 'Y88', 'M8251', 'M9560', 'M9180', 'M9823', 'M9380', 'M9091', 'M9800']
# 创建布尔索引
mask = Full_data['ICD10_CODE'].str.startswith(tuple(del_icd))

# 应用布尔索引删除匹配条件的行
Full_data = Full_data[~mask]
Full_data

,SUBJECT_ID,HADM_ID,ADMITTIME,ICD10_CODE,Sort
0,1,0,2022-03-05,G47.9,0
1,2,1,2018-10-29,I67.8,0
2,3,2,2020-04-09,J18.9,0
3,3,3,2020-05-13,F01.1,0
4,3,4,2020-06-12,J69.0,0
...,...,...,...,...,...
4351675,102408,348534,2022-12-07,None,15
4351676,102408,348535,2022-12-19,None,15
4351677,102408,348536,2023-01-13,None,15
4351678,102408,348537,2023-01-31,None,15


In [13]:
Full_data.replace('None', np.nan, inplace=True)
Full_data

,SUBJECT_ID,HADM_ID,ADMITTIME,ICD10_CODE,Sort
0,1,0,2022-03-05,G47.9,0
1,2,1,2018-10-29,I67.8,0
2,3,2,2020-04-09,J18.9,0
3,3,3,2020-05-13,F01.1,0
4,3,4,2020-06-12,J69.0,0
...,...,...,...,...,...
4351675,102408,348534,2022-12-07,NaN,15
4351676,102408,348535,2022-12-19,NaN,15
4351677,102408,348536,2023-01-13,NaN,15
4351678,102408,348537,2023-01-31,NaN,15


In [14]:
# 删除'ICD10_CODE'列为空的行
Full_data.dropna(subset=['ICD10_CODE'])
# 新增列seq_num表示新的排序顺序
Full_data['SEQ_NUM'] = Full_data.groupby(['SUBJECT_ID', 'ADMITTIME']).cumcount() + 1
# 根据id、ADMITTIME和Sort排序
Full_data = Full_data.sort_values(['SUBJECT_ID', 'ADMITTIME', 'SEQ_NUM'])

# 删除'ICD10_CODE'列为空的行
Full_data = Full_data.dropna(subset=['ICD10_CODE'])
# 删除列Sort
Full_data.drop(columns=['Sort'], inplace=True)

print(Full_data.shape)

(1315701, 5)


In [15]:
Adm_.to_csv('../data/raw/ADMISSIONS.csv', index=False)
Full_data.to_csv('../data/raw/DIAGNOSES_ICD.csv', index=False)

In [16]:
id_counts = Adm_['SUBJECT_ID'].value_counts()
# 输出counts大于1的人数
result = id_counts[id_counts > 1].index
print(len(id_counts), len(result))

102309 43401


In [17]:
Adm_

,SUBJECT_ID,HADM_ID,record_num,ADMITTIME,bir,age,gender,ocp,mary,pri_code
0,1,0,5046540.0,2022-03-05,1974-04-16,47,2,27.0,20.0,G47.9
1,2,1,1678982.0,2018-10-29,1968-06-06,50,2,17.0,20.0,I67.8
2,3,2,None,2020-04-09,1938-09-03,81,1,80.0,20.0,J18.9
3,3,3,46188.0,2020-05-13,1938-09-03,81,1,80.0,20.0,F01.1
4,3,4,57492.0,2020-06-12,1938-09-03,81,1,80.0,20.0,J69.0
...,...,...,...,...,...,...,...,...,...,...
348534,102408,348534,5712574.0,2022-12-07,1931-01-01,91,2,80.0,20.0,I63.3
348535,102408,348535,5735758.0,2022-12-19,1931-01-01,91,2,80.0,20.0,F41.1
348536,102408,348536,5804901.0,2023-01-13,1931-01-01,92,2,80.0,20.0,J15.8
348537,102408,348537,5932560.0,2023-01-31,1931-01-01,92,2,80.0,20.0,F01.2


In [18]:
Full_data

,SUBJECT_ID,HADM_ID,ADMITTIME,ICD10_CODE,SEQ_NUM
0,1,0,2022-03-05,G47.9,1
271980,1,0,2022-03-05,K29.5,2
543960,1,0,2022-03-05,F45.3,3
815940,1,0,2022-03-05,K59.0,4
1,2,1,2018-10-29,I67.8,1
...,...,...,...,...,...
1087919,102409,348538,2022-09-01,F41.1,4
1359899,102409,348538,2022-09-01,G47.9,5
1631879,102409,348538,2022-09-01,E78.5,6
1903859,102409,348538,2022-09-01,M48.9,7
